title: MCU insights
author: Vladas Jankus 
date: 2021-12-06
region: EU  
tags: cards, card, MCU, product, ftMAU, lapse 
summary: The reasearch looks at Monthly Cards Users (MCU). Provides basic insights into the new metric, looks at MCU conversion, compares MCU against MAU.

In [1]:
cd /app/

/app


In [2]:
!pip install altair
import altair as alt
import pandas as pd
from utils.datalib_database import df_from_sql
from IPython.display import HTML, Markdown as md

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [3]:
sql_path = "research/product/cards/20211206_mcu_insights/queries/"

In [4]:
# load an example of MCU customer
query = open(sql_path + "customer_example_1.sql", "r").read()
customer_example_1 = df_from_sql("redshiftreader", query)

In [5]:
# load customer lifespan vs MCU comparison
query = open(sql_path + "cst_lifetime_vs_mcu.sql", "r").read()
cst_lifetime_vs_mcu = df_from_sql("redshiftreader", query)

In [6]:
# load MCU by user type on lifespan months
query = open(sql_path + "mcu_by_user_type.sql", "r").read()
mcu_by_user_type = df_from_sql("redshiftreader", query)

In [7]:
# load MCU vs MAU monthly comparison
query = open(sql_path + "ftmau_mcu_mau_monthly.sql", "r").read()
ftmau_mcu_mau = df_from_sql("redshiftreader", query)

In [8]:
# load MCU conversion
query = open(sql_path + "mcu_conversion.sql", "r").read()
mcu_conversion = df_from_sql("redshiftreader", query)

In [9]:
# load active time percentage
query = open(sql_path + "active_time_perc.sql", "r").read()
active_time_perc = df_from_sql("redshiftreader", query)

In [10]:
# load active time over past 3 years
query = open(sql_path + "active_time_past_3yrs.sql", "r").read()
active_time_past_3yrs = df_from_sql("redshiftreader", query)

In [11]:
# load MCU vs MAU activity lenght comparison
query = open(sql_path + "activity_length_mau_mcu.sql", "r").read()
mcu_vs_mau_activity_lenght = df_from_sql("redshiftreader", query)

In [12]:
# load MCU lapse influence on MAU
query = open(sql_path + "lapse_influence.sql", "r").read()
mcu_influence = df_from_sql("redshiftreader", query)

# MCU Insights
Vladas Jankus<br/>
2021-11-11

In this research we will take a look at the newly created MCU metric. We will cover 4 topics:

1. Basic introduction to MCU
2. MCU conversion
3. Lifetime MCU rate
4. MCU influence on MAUs

Each of the topics will give some basic insights, and hopefully will provide some grounds for a more deeper research. TL;DR section will summarize findings on each of the topics. Then all four section titles will be numbered accordingly, you can use the index below for a quick jump into the topic of your interest.

If you have any feedback or ideas (or see mistakes), I would love it if you could [Slack](https://n26.slack.com/archives/D8CGKC1MZ) me.

### Contents:
* [TL;DR](#tldr)
* [1. Basic introduction to MCU](#1)
  * [1.1. Term definitions](#11)
  * [1.2. Customer example](#12)
  * [1.3. Activity periods](#13)
  * [1.4. MCU numbers](#14)
* [2. MCU conversion](#2)
  * [2.1. MCU 10-week conversion rate](#21)
  * [2.2. ftMAU and MCU customer volumes](#22)
* [3. Lifetime MCU rate](#3)
  * [3.1. MCU progression over customer lifetime](#31)
  * [3.2. MCU progression by customer type](#32)
  * [3.3. Percentage of time spent as MCU](#33)
* [4. MCU influence on MAUs](#4)
  * [4.1. Historic MAU and MCU numbers](#41)
  * [4.2. MCU influence on MAU length](#42)
  * [4.3. MCU influence on MAU lapse](#43)
* [5. Conclusion](#5) 

## TL;DR <a name="tldr"></a>
This research provides a quick look into MCU insights and provides some ground for further research. We have looked into four main topics that are summarized below.

1. **Basic introduction to MCU.** Monthly cards user (MCU) is a very similar metric to monthly active user (MAU). The only difference being that only card transactions are counted. With MCU we can see customers, who are making at least one transaction in 35 days. At the moment we have around 1,6 million MCU customers, with a yearly growth of around 30%.
2. **MCU conversion.** MCU conversion rate is calculated from customers who have been first time MAUs (ftMAU). ftMAU means that they have made a topup and became ready to use their card. Recently around 75%-70% of customers convert to MCUs in 60 days. However quite many of them lapse, and we have around 52% ftMAU to MCU long-term ratio.
3. **Lifetime MCU rate.** When looking at ftMAU/MCU ratio over customer lifetime, we notice at 2 months after becoming ftMAU 76% of customers are MCUs. This number drops to 66% by the 5th month and to 56% after 2 years. We can identify a group of customers that lapse quickly within 4 months, and then another group who gradually lapse within 2 years.
4. **MCU influence on MAUs.** This may sound obvious, but MCUs are the main driving force for MAUs. Around 80% of MAUs are also MCUs. We can also see that the ones who are only MAUs have much shorter activity times - proving that card usage creates stronger MAU engagement. Lastly we found out that around 70% of MAUs lapse is caused by lapsing MCUs.


## 1. Basic introduction to MCU <a name="1"></a>
Monthly card user (MCU) is a metric that displays customers who are active card users on a monthly basis. The metric can be directly comparable with monthly active user (MAU) because it is built with the same parameters. Anyone who understands the MAU concept should find MCU easy, because it's exactly the same, but just limited to card transactions.

### 1.1. Term definitions <a name="11"></a>

- **MCU** - monthly card user. A customer who has made at least one card transaction in the past 35 days.
- **ftMAU** - first time MAU. A customer who topped up. First time MAU is always a topup of the account. This is important in this context, because a customer who topped up becomes ready to use their card.
- **Activity period** - a stretch of time, where the customer had an MCU status. During this time the customer must have at least one card transaction every 35 days. Note that the activity period ends 35 days after the last transaction.
- **Card transaction** - a card transaction for MCU is an Authorisation Attempt (AA). We take AAs because of several reasons: AA shows customer agreement to pay, therefore marks activity; AA timestamp is closer to the actual transaction time than PT; MAU is also based on AA transactions, so it allows equal comparison of these two.
- **MCU/lifespan ratio** - the percentage of time one customer spends with an MCU status.

### 1.2. Customer example <a name="12"></a>
Below you can see one example of a customer and his MCU status. Orange color indicates weeks when the customer was an MCU. Green bars display the card transactions this customer made. This chart spans during the whole lifetime of the customer - (s)he joined N26 on week 2019-41 and made their first card transaction on week 2019-51.

In [13]:
base = alt.Chart(customer_example_1).encode(x=alt.X("week:O", title="Week"))

activity_bar = base.mark_bar(color="#f4a261", opacity=0.2).encode(
    y=alt.Y("mcu:Q", axis=None)
)

tx_bar = base.mark_bar(color="#2a9d8f", opacity=1).encode(
    y=alt.Y("card_transactions:Q", title="Card transactions")
)


alt.layer(tx_bar, activity_bar).properties(
    width=800, height=200, title="One example MCU customer: weeks since ftMAU"
).resolve_scale(y="independent")

alt.LayerChart(...)

The main purpose of this chart is to display how MCU status is maintained over 35 day stretches. Note that 35 days is exactly 5 weeks. During weeks 2020-04 and 2020-14 the customer made only one transaction, but because of the 35 day period, the customer maintained MCU status until week 2020-14. Then it stopped as the customer was not making any other transactions in the next half-year.

As of this date, our example customer is an MCU since 2020-53. There are some card transactions every other week, but all of them are within 35 days, so the activity is continuous throughout the whole year 2021.

In this chart we can also visualize the MCU/lifespan ratio, which is 63% for this customer (percentage is not visible on the chart). It is the area covered in orange - the amount of lifetime spent with an MCU status.

### 1.3. Activity periods <a name="13"></a>
Activity periods (orange time spans in the chart above) are an important part of the concept. If we are seeking to increase card usage and reduce lapses, MCU periods should get longer and more continuous. Ideally covering the whole lifespan of the customer. Lapsing card usage might also be not the worst situation, as long as the customer maintains a high MCU/lifespan ratio.

### 1.4. MCU numbers <a name="14"></a>
In the introduction we can also quickly see how the MCU numbers look. The chart below displays how MCU numbers have changed over time in the past 3 years.

In [14]:
ftmau_mcu_mau = ftmau_mcu_mau.rename(
    columns={"ftmau": "ftMAU", "mcus": "MCU", "maus": "MAU"}
)
base = alt.Chart(ftmau_mcu_mau).encode(x=alt.X("month:T", title="Month"))

mcu_area = base.mark_area(opacity=0.6, color="#2a9d8f").encode(
    y=alt.Y("MCU:Q", title="MCUs")
)
nearest = alt.selection(
    type="single", nearest=True, on="mouseover", fields=["month"], empty="none"
)
selectors = (
    base.mark_point()
    .encode(
        alt.X("month:T"),
        opacity=alt.value(0),
    )
    .add_selection(nearest)
)
points_mcu_area = base.mark_point(color="#2a9d8f").encode(
    y=alt.Y("MCU:Q"), opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)
text_mcu_area = mcu_area.mark_text(align="left", dx=5, dy=-5).encode(
    y=alt.Y("MCU:Q"), text=alt.condition(nearest, "MCU:Q", alt.value(" "), format=",")
)
rules = (
    base.mark_rule(color="gray")
    .encode(
        x="month:T",
    )
    .transform_filter(nearest)
)
alt.layer(mcu_area, selectors, points_mcu_area, text_mcu_area, rules).properties(
    width=700, height=300, title="MCU over time"
)

alt.LayerChart(...)

Currently we have around 1,6 million MCUs. The number has been steadily increasing over the past three years. With an exception during the first Covid quarantine in Q2 of 2020. Since the beginning of 2021 the number of MCUs has increased by 33%, from 1,2 million to 1,6 million.

## 2. MCU conversion <a name="2"></a>
The fact that the MCU numbers have been steadily growing alone does not really tell us anything. Our customer base has seen a lot of growth and higher numbers of MCU are mostly a consequence of that. Therefore we need to look at the MCU conversion rate from ftMAU customers.

Why do we take conversion rate from first time MAUs (ftMAU) and not KYC completed (KYCc) customers? Because customers who become first time MAUs are fully ready to use their card and become MCUs. Many customers complete KYC and then don't top up their accounts - this factor does not fall into the cards area. A customer becomes ready to use his card when he has funds in the account, and the topup makes that customer ftMAU. Therefore looking at conversion from ftMAU to MCU makes most sense.

Note: in the past we have seen an error, where customers can get a virtual card before the first topup. This resulted in many customers who owned a virtual card without becoming ftMAU. Nevertheless, this error does not interfere without our logic because owning a card does not make a customer MCU. To make a card transaction and become ftMCU the customer still needs to top up their account first, so ftMAU will always happen before ftMCU.

### 2.1. MCU 10-week conversion rate <a name="21"></a>
Now we can take a 10 week conversion rate. We are taking 10 weeks because it is known that most customers start using the card 2 months after ftMAU, this is discussed later in the research. 10 weeks is slightly more than 2 months.

Chart below displays the percentage of customers who convert from ftMAU to MCU in 10 weeks. X-axis is the month when the customer became ftMAU, and yellow bars display the count of ftMAUs in the cohort. You can see the percentage of customers who became MCU in 10 weeks.

In [15]:
base = alt.Chart(mcu_conversion).encode(x=alt.X("month:O", title="Month"))

bar = base.mark_bar(opacity=0.6, color="#f4a261").encode(
    y=alt.Y("ftmau_customers:Q", title="ftMAU customers")
)
line = base.mark_line(color="#2a9d8f").encode(
    y=alt.Y("converted_to_mcu:Q", axis=alt.Axis(format="%"), title="% converted to MCU")
)
nearest = alt.selection(
    type="single", nearest=True, on="mouseover", fields=["month"], empty="none"
)
selectors = (
    base.mark_point()
    .encode(
        alt.X("month:O"),
        opacity=alt.value(0),
    )
    .add_selection(nearest)
)
points_bar = base.mark_point(color="#f4a261").encode(
    y=alt.Y("ftmau_customers:Q", axis=None),
    opacity=alt.condition(nearest, alt.value(1), alt.value(0)),
)
points_line = base.mark_point(color="#2a9d8f").encode(
    y=alt.Y("converted_to_mcu:Q", axis=None),
    opacity=alt.condition(nearest, alt.value(1), alt.value(0)),
)
text_bar = bar.mark_text(align="left", dx=5, dy=-5).encode(
    y=alt.Y("ftmau_customers:Q", axis=None),
    text=alt.condition(nearest, "ftmau_customers:Q", alt.value(" "), format=","),
)
text_line = line.mark_text(align="left", dx=5, dy=-5).encode(
    y=alt.Y("converted_to_mcu:Q", axis=None),
    text=alt.condition(nearest, "converted_to_mcu:Q", alt.value(" "), format=".2%"),
)
rules = (
    base.mark_rule(color="gray")
    .encode(
        x="month:O",
    )
    .transform_filter(nearest)
)
alt.layer(
    bar, line, selectors, points_bar, points_line, text_bar, text_line, rules
).properties(
    width=700, height=300, title="ftMAU→MCU conversion in 10 weeks"
).resolve_scale(
    y="independent"
)

alt.LayerChart(...)

There are several takeaways from this chart (note that 2021 October/November numbers are not complete):

The 10 week conversion rate is lower than in 2019. Reasons are unknown, could be related with less card transactions due to the Covid crisis, but this requires more investigation. 

There is a visible increase of the conversion in 2021-Q3. It also coincides with more ftMAU customers in the same months. Reasons are also unknown and would be interesting to look at.

### 2.2. ftMAU and MCU customer volumes <a name="22"></a>
This was the 10 week conversion rate. However many of these customers who convert to MCU later lapse and become inactive. A chart below displays how many ftMAU and MCU customers we have each month since 2019. This chart can be treated as long-term conversion, including the customers who lapsed.

Yellow area is the count of customers who have ftMAU status each month, and green is the count of customers who are MCUs. Red line indicates the percentage of MCUs among ftMAU customers.

In [16]:
base = alt.Chart(ftmau_mcu_mau).encode(x=alt.X("month:T", title="Month"))

area = (
    base.transform_fold(["ftMAU", "MCU"])
    .mark_area(interpolate="basis", opacity=0.4)
    .encode(
        y=alt.Y("value:Q", stack=None, title="# of ftMAU and MCU"),
        color=alt.Color(
            "key:N",
            scale=alt.Scale(domain=["ftMAU", "MCU"], range=["#f4a261", "#2a9d8f"]),
        ),
    )
)
line = base.mark_line(color="#e76f51", interpolate="basis").encode(
    y=alt.Y("perc_on_ftmau:Q", axis=alt.Axis(format="%"), title="Percentage of MCUs")
)

# Create a selection that chooses the nearest point & selects based on x-value
nearest = alt.selection(
    type="single", nearest=True, on="mouseover", fields=["month"], empty="none"
)
# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = (
    base.mark_point(interpolate="basis")
    .encode(
        alt.X("month:T"),
        opacity=alt.value(0),
    )
    .add_selection(nearest)
)

# Draw points on the line, and highlight based on selection
points_area = (
    base.transform_fold(["ftMAU", "MCU"])
    .mark_point()
    .encode(
        y=alt.Y("value:Q", axis=None),
        color="key:N",
        opacity=alt.condition(nearest, alt.value(1), alt.value(0)),
    )
)

points_line = line.mark_point(interpolate="basis", color="#e76f51").encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)
# Draw text labels near the points, and highlight based on selection
text_area = area.mark_text(align="left", dx=5, dy=-5).encode(
    y=alt.Y("value:Q", axis=None),
    text=alt.condition(nearest, "value:Q", alt.value(" "), format=","),
)
text_line = line.mark_text(align="left", dx=5, dy=-5).encode(
    y=alt.Y("perc_on_ftmau:Q", axis=None),
    text=alt.condition(nearest, "perc_on_ftmau:Q", alt.value(" "), format=".2%"),
)
# Draw a rule at the location of the selection
rules = (
    base.mark_rule(color="gray")
    .encode(
        x="month:T",
    )
    .transform_filter(nearest)
)

alt.layer(
    area, line, selectors, points_area, points_line, text_area, text_line, rules
).properties(
    width=700, height=300, title="ftMAU and MCU monthly numbers"
).resolve_scale(
    y="independent"
)

alt.LayerChart(...)

In the first place, we can notice that the percentage of MCUs is much lower than in 10 week conversion. This means that in 10 weeks, around 75% of customers become MCUs, but in the long term only around 52% of ftMAU customers are MCUs. Also it appears that the ratio of MCUs has been going down over the past 3 years, but the trend in 2021 is more stable. Most recently in 2021 October, 52% of ftMAU customers were MAUs, while in February this number was 51%. 


## 3. Lifetime MCU rate <a name="3"></a>
If the above assumption that many customers lapse right after converting is correct, we should spot this by looking at the customer lifetime status. Lifetime in this case starts with becoming ftMAU until the account closure. We don't consider customers who have not been ftMAUs because they never had an opportunity to become MCUs.

### 3.1. MCU progression over customer lifetime <a name="31"></a>
The chart below shows what percentage of customers are MCUs at X months since they became ftMAU. X-axis shows the months since ftMAU, yellow color displays the count of ftMAU customers who reached x months, green color displays the count of customers who reached x months with an MCU status. Red line is the rate between these two at x months since ftMAU.

In [17]:
base = alt.Chart(cst_lifetime_vs_mcu).encode(
    x=alt.X("lifetime_month:O", title="Month since ftMAU")
)

active_area = base.mark_area(opacity=0.6, color="#f4a261").encode(
    y=alt.Y(
        "active_customers:Q",
        scale=alt.Scale(domain=[0, 5000000]),
        title="# of ftMAU or MCU customers",
    )
)
mcu_area = base.mark_area(opacity=0.6, color="#2a9d8f").encode(
    y=alt.Y("mcu:Q", scale=alt.Scale(domain=[0, 5000000]), axis=None)
)
line = base.mark_line(color="#e76f51").encode(
    y=alt.Y("mcu_rate:Q", axis=alt.Axis(format="%"), title="Percentage of MCUs")
)
nearest = alt.selection(
    type="single", nearest=True, on="mouseover", fields=["lifetime_month"], empty="none"
)
selectors = (
    base.mark_point()
    .encode(
        alt.X("lifetime_month:O"),
        opacity=alt.value(0),
    )
    .add_selection(nearest)
)
points_active_area = base.mark_point(color="#f4a261").encode(
    y=alt.Y("active_customers:Q", axis=None, scale=alt.Scale(domain=[0, 5000000])),
    opacity=alt.condition(nearest, alt.value(1), alt.value(0)),
)
points_mcu_area = base.mark_point(color="#2a9d8f").encode(
    y=alt.Y("mcu:Q", axis=None, scale=alt.Scale(domain=[0, 5000000])),
    opacity=alt.condition(nearest, alt.value(1), alt.value(0)),
)
points_line = base.mark_point(color="#e76f51").encode(
    y=alt.Y("mcu_rate:Q", axis=None),
    opacity=alt.condition(nearest, alt.value(1), alt.value(0)),
)
text_active_area = active_area.mark_text(align="left", dx=5, dy=-5).encode(
    y=alt.Y("active_customers:Q", axis=None, scale=alt.Scale(domain=[0, 5000000])),
    text=alt.condition(nearest, "active_customers:Q", alt.value(" "), format=","),
)
text_mcu_area = mcu_area.mark_text(align="left", dx=5, dy=-5).encode(
    y=alt.Y("mcu:Q", axis=None, scale=alt.Scale(domain=[0, 5000000])),
    text=alt.condition(nearest, "mcu:Q", alt.value(" "), format=","),
)
text_line = line.mark_text(align="left", dx=5, dy=-5).encode(
    y=alt.Y("mcu_rate:Q", axis=None),
    text=alt.condition(nearest, "mcu_rate:Q", alt.value(" "), format=".2%"),
)
rules = (
    base.mark_rule(color="gray")
    .encode(
        x="lifetime_month:O",
    )
    .transform_filter(nearest)
)
alt.layer(
    active_area,
    mcu_area,
    line,
    selectors,
    points_active_area,
    points_mcu_area,
    points_line,
    text_active_area,
    text_mcu_area,
    text_line,
    rules,
).properties(
    width=700, height=300, title="MCU ratio in X months since ftMAU"
).resolve_scale(
    y="independent"
)

alt.LayerChart(...)

In the first month we see only 54% of MCUs, however data is based on the calendar month and in many cases it's not full. By 2nd month 76% of customers are MCUs and the ratio starts declining until it settles around 60%-57% after 1 year of tenure. After 2.5 years with N26 the ratio of MCUs starts gradually climbing again, mostly because inactive customers drop off and close their accounts.

What is noticeable in this chart is the initial drop of customers. It looks like our assumption from the previous section was correct and there is a high number of customers who become MAUs in the first 60 days but then lapse.

There probably are at least four different groups of MCU customers that have different properties:
- Currently inactive MCUs
  1. Customers who start using their card but stop within 4 months and don't return. Potentially they are the reason for the initial drop.
  2. Customers who use the card for 4+ months but lapse.
- Currently active MCUs
  3. Customers who have only 1 continuous activity.
  4. Customers who lapsed>returned at least once, but are currently active
  
### 3.2. MCU progression by customer type <a name="32"></a>
All these four groups are displayed in a chart below. The data is ordered on the X axis based on months since the ftMAU date (just like in the chart above). The first chart gives absolute customer numbers and the second one is normalized to better see the customer type distribution at every lifetime month.

In [18]:
base = alt.Chart(mcu_by_user_type).encode(
    x=alt.X("lifetime_month:O", title="Month since ftMAU")
)
area = (
    base.mark_area(opacity=0.6)
    .encode(y=alt.Y("customers:Q", stack="zero", title="MCUs"), color="mcu_type")
    .properties(
        width=700,
        height=300,
        title="MCU customers by lapse situation, months since ftMAU",
    )
)
area_stacked = (
    base.mark_area(opacity=0.6)
    .encode(
        y=alt.Y(
            "customers:Q",
            axis=alt.Axis(format="%"),
            stack="normalize",
            title="Percent of MCUs",
        ),
        color="mcu_type",
    )
    .properties(width=700, height=150)
)

alt.vconcat(area, area_stacked)

alt.VConcatChart(...)

We can see that the spike volume is mostly consisting of customers who lapsed in 4 months. Some of them might return using the card in the future, but in general there is a group of customers that lapse right after starting.

At the bottom of the chart, in teal color, you can see a group of customers who have one continuous activity. At every point in the lifetime these make up around 25%-30% of MCU customers.

In the middle of the chart, orange and red colors, are the customers who lapse and return. The only difference is that the orange group of customers lapsed and are not currently active while the red area shows customers who returned and are currently active. These customers make up the highest share of MCUs.

### 3.3. Percentage of time spent as MCU <a name="33"></a>
Looking at customers based on lapses might not be the best way to evaluate engagement. We don't expect all the customers to use our card as the primary, so lapsing and returning might be a normal part of the lifecycle. Also, if we focus on reducing lapsers, this will turn into customers spending more time in MCU status. Therefore the percentage of time spent as MCU may be the target metric.

Lifetime in this case would mean the duration since ftMAU - when the customer is completely ready to engage in card transactions, until the account closure - just like in the previous sections.

The chart below displays all customers split by the amount of lifetime they spent with MCU status. Colors show the tenure length - just to evaluate whether higher MCU rates correlate with longer lifetime.

Please note that we count customers who had MCU status at least once. We ignore ftMAU customers who never were MCUs.

In [19]:
base = alt.Chart(active_time_perc).encode(
    x=alt.X("time_as_mcu:O", title="Percent of time spent as MCU")
)
# Create a bar for activity periods
bar = base.mark_bar(opacity=0.8).encode(
    y=alt.Y(
        "customer_perc:Q",
        stack="zero",
        axis=alt.Axis(format="%"),
        title="Percentage of MCUs",
    ),
    color=alt.Color("cst_tenure:N"),
)
# Create a selection that chooses the nearest point & selects based on x-value
nearest = alt.selection(
    type="single", nearest=True, on="mouseover", fields=["time_as_mcu"], empty="none"
)
# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = (
    base.mark_point()
    .encode(
        alt.X("time_as_mcu:O"),
        opacity=alt.value(0),
    )
    .add_selection(nearest)
)
text = bar.mark_text(align="center", dx=2, dy=-5).encode(
    y=alt.Y("customer_perc:Q", stack="zero"),
    text=alt.condition(nearest, "customer_perc:Q", alt.value(" "), format=".0%"),
)
alt.layer(bar, selectors, text).properties(
    width=700,
    height=300,
    title="MCUs split by percentage of MCU status on lifetime since ftMAU",
)

alt.LayerChart(...)

It is interesting to see that almost 25% of customers spent over 90% of their lifetime with an MCU status. More analysis would be required here, but we can assume that healthy card users spend over 80% of their lifetime with an MCU status. 

The rest of the customers could be targeted with initiatives to prevent their lapsing. Preventing MCU lapse would eventually increase MCU/lifetime ratio. 

Just for curiosity, we can see how this split of customers changed historically. See chart below.

In [20]:
base = alt.Chart(active_time_past_3yrs).encode(x=alt.X("month:T", title="Month"))
area = base.mark_area(opacity=0.8).encode(
    y=alt.Y("customers:Q", stack="normalize", title="MCUs"), color="time_as_mcu:N"
)
area.properties(width=700, height=300, title="MCUs by time as MCU historically")

alt.Chart(...)

It appears that in more recent months we have more customers who spend less time with an MCU status. This change is very slight, but could be an interesting point for further investigations. However, it could be expected that by making less frequent MCU lapses, we could increase the percentage of high MCU/lifetime customers.

## 4. MCU influence on MAUs <a name="4"></a>

Card usage is obviously one of the main driving factors for MAU customers. However this topic was not quantified and we were not really sure how important continuous card transactions are for MAU. In this section we will be looking at how MCU periods influence MAU.

MCU and MAU are directly comparable, because they are constructed with the same parameters. The only difference is that MCU only accounts for card transactions, while MAU accounts for all transactions.

### 4.1. Historic MAU and MCU numbers <a name="41"></a>
Monthly MAU and MCU numbers are visualized in the chart below. Blue and orange areas display MAU and MCU numbers, while the red line shows the percentage of MCUs within MAUs.

In [21]:
base = alt.Chart(ftmau_mcu_mau).encode(x=alt.X("month:T", title="Month"))

area = (
    base.transform_fold(["MAU", "MCU"])
    .mark_area(interpolate="basis", opacity=0.4)
    .encode(y=alt.Y("value:Q", stack=None, title="# of MCU or MAU"), color="key:N")
)
line = base.mark_line(color="red", interpolate="basis").encode(
    y=alt.Y("perc_on_mau:Q", axis=alt.Axis(format="%"), title="Percentage of MCU")
)

# Create a selection that chooses the nearest point & selects based on x-value
nearest = alt.selection(
    type="single", nearest=True, on="mouseover", fields=["month"], empty="none"
)
# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = (
    base.mark_point(interpolate="basis")
    .encode(
        alt.X("month:T"),
        opacity=alt.value(0),
    )
    .add_selection(nearest)
)

# Draw points on the line, and highlight based on selection
points_area = (
    base.transform_fold(["MAU", "MCU"])
    .mark_point(color="black", interpolate="basis")
    .encode(
        y=alt.Y("value:Q", axis=None),
        color="key:N",
        opacity=alt.condition(nearest, alt.value(1), alt.value(0)),
    )
)

points_line = line.mark_point(interpolate="basis", color="red").encode(
    alt.Y("value:Q", axis=None),
    opacity=alt.condition(nearest, alt.value(1), alt.value(0)),
)
# Draw text labels near the points, and highlight based on selection
text_area = area.mark_text(align="left", dx=5, dy=-5).encode(
    y=alt.Y("value:Q", axis=None),
    text=alt.condition(nearest, "value:Q", alt.value(" "), format=","),
)
text_line = line.mark_text(align="left", dx=5, dy=-5).encode(
    y=alt.Y("perc_on_mau:Q", axis=None),
    text=alt.condition(nearest, "perc_on_mau:Q", alt.value(" "), format=".2%"),
)
# Draw a rule at the location of the selection
rules = (
    base.mark_rule(color="gray")
    .encode(
        x="month:T",
    )
    .transform_filter(nearest)
)

alt.layer(
    area, line, selectors, points_area, points_line, text_area, text_line, rules
).properties(width=750, height=300, title="MCU and MAU comparison").resolve_scale(
    y="independent"
)

alt.LayerChart(...)

The percentage of MAUs who are card users is stable and around 80%. The only fluctuation we see in the Q2 of 2020, when the first Covid quarantine happened. In October we had 1.9 million MAUs, out of them, 1.6 million were MCUs.

The chart above suggests that monthly active users (MAUs) are driven by monthly card users (MCUs). This is because such a high volume of MAUs are also MCUs at the same time.

### 4.2. MCU influence on MAU length <a name="42"></a>
To further investigate this point we can check whether customers who are only MAUs have different MAU status lengths. The assumption here is, that card usage creates more committed customers, who retain their MAU status for longer periods. Let's look at the chart below, data here is taken from the customers who were active in 2021. Color coding is based on the length of MAU status.

In [22]:
bars = (
    alt.Chart(mcu_vs_mau_activity_lenght)
    .mark_bar(opacity=0.8)
    .encode(
        x=alt.X("sum(percentage):Q", stack="zero", axis=alt.Axis(format="%"), title=""),
        y=alt.Y("no_card_mau:N"),
        color=alt.Color("activity_length:N", title="Activity Length"),
        order=alt.Order(
            # Sort the segments of the bars by this field
            "activity_length:N",
            sort="ascending",
        ),
    )
)

text = (
    alt.Chart(mcu_vs_mau_activity_lenght)
    .mark_text(dx=-17, dy=3, color="white")
    .encode(
        x=alt.X("sum(percentage):Q", stack="zero"),
        y=alt.Y("no_card_mau:N", axis=alt.Axis(title="")),
        detail="activity_length:N",
        order=alt.Order(
            # Sort the segments of the bars by this field
            "activity_length:N",
            sort="ascending",
        ),
        text=alt.Text("sum(percentage):Q", format=".1%"),
    )
)

(bars + text).properties(
    width=700, height=150, title="Lenght of MAU activity for MAUs vs MAUs+MCUs"
)

alt.LayerChart(...)

From the chart above we can see that monthly card users (MCU) are a good indication for engaged, long term MAUs. Monthly active users, who have no card activity have significantly shorter activity periods.

Nearly 70% of customers who were MAUs without card activity, lasted only for 1 month. 90% of such customers lapse their MAU status after 2 months.

MAUs who are also MCUs have significantly longer activity periods. 45% of time their MAU activity stretches over 3 months.

### 4.3. MCU influence on MAU lapse <a name="43"></a>
In the context that we currently have a ceiling of 50k KYC completed customers in a month, controlling MAU lapses becomes very important. There is an assumption that customers who lapse as MCUs also drag down their MAU status. This would be another example of MCUs as the driving force for MAUs.

Let's look at the MAU lapsers who are influenced by MCU lapsers. The chart below shows the monthly percentage of MAUs, who lapsed right after they lapsed as MCUs. In other words - this is the percentage of MAUs who lapsed within one day of their last card transaction.  

In [23]:
line = (
    alt.Chart(mcu_influence)
    .mark_line(interpolate="basis")
    .encode(
        x=alt.X("mau_end:T", title="MAU lapse month"),
        y=alt.Y(
            "caused_by_mcu_lapse:Q",
            axis=alt.Axis(format="%"),
            title="Percent of MAU lapse caused by MCU",
        ),
    )
)
# Create a selection that chooses the nearest point & selects based on x-value
nearest = alt.selection(
    type="single", nearest=True, on="mouseover", fields=["mau_end"], empty="none"
)
# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = (
    line.mark_point(interpolate="basis")
    .encode(
        alt.X("mau_end:T"),
        opacity=alt.value(0),
    )
    .add_selection(nearest)
)
points = line.mark_point(interpolate="basis").encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)
text = line.mark_text(align="left", dx=5, dy=-5).encode(
    text=alt.condition(nearest, "caused_by_mcu_lapse:Q", alt.value(" "), format=".2%")
)
# Draw a rule at the location of the selection
rules = (
    line.mark_rule(color="gray")
    .encode(
        x="mau_end:T",
    )
    .transform_filter(nearest)
)

alt.layer(line, selectors, points, text, rules).properties(
    width=700, height=300, title="MAU lapsers caused by MCU lapse"
)

alt.LayerChart(...)

Around 70% of MAU lapse is caused by MCU lapse. These customers stop being MAUs because they stopped their card transactions. The rest of 30% lapsed without being MCUs at all, or stopped card transactions earlier, and did not lapse immediately within 1 day. It is interesting to note, that over the past two years this trend seems to be slightly moving down. 

Nevertheless, it shows that card users are not only an important factor of driving the MAUs forward, but also an important factor for holding it down.

## 5. Conclusion <a name="5"></a>

MCU can be a good tool to identify gaps in user engagement. We are now able to quantify card usage influence on MAUs, and we can see that there is a high impact. Given the current situation, that we have a ceiling of new KYCc customers, making current customers better engaged becomes a very important topic. This research work looked at several topics, and none of them too deeply, so there should be some followup work done.

We should aim for increasing customer MCU/lifetime ratio - increase the percentage of time our customers are in MCU status. Knowing that ftMAU>MCU conversion is quite high, this can be done by working on the factors that make MCUs lapse. This should be one of the followup topics.

We also haven't looked at the transactional side of MCUs. We did not investigate MCU behaviour based on their different transactional use - like different types of MCCs; wallet or transaction channel preferences; even the frequency of transactions. This is another broad area that closely relates to MCUs and should be followed up. 